# Tutorial 4 : The lattice-Boltzmann method in ESPResSo - Part 3

### 5.2 Step 2: Diffusion of a polymer

One of the typical applications of **ESPResSo** is the simulation of polymer chains with a bead-spring-model. For this we need a repulsive interaction between all beads, for which one usually takes a shifted and truncated Lennard-Jones (so-called Weeks–Chandler–Andersen or WCA) interaction, and additionally a bonded interaction between adjacent beads to hold the polymer together. You have already learned that the command

```python
system.non_bonded_inter[0, 0].lennard_jones.set_params(
    epsilon=1.0, sigma=1.0, shift=0.25, cutoff=1.1225)
```

creates a Lennard-Jones interaction with $\varepsilon=1.$, $\sigma=1.$,
$r_\text{cut} = 1.1225$ and $\varepsilon_\text{shift}=0.25$ between particles
of type 0, which is the desired repulsive interaction. The command

```python
fene = espressomd.interactions.FeneBond(k=7, r_0=1, d_r_max=2)
```

creates a <tt>FeneBond</tt> object (see **ESPResSo** manual for the details). What is left to be done is to add this bonded interaction to the system via

```python
system.bonded_inter.add(fene)
```

and to apply the bonded interaction to all monomer pairs of the polymer as shown in the script below.

**ESPResSo** provides a function that tries to find monomer positions that minimize the overlap between
monomers of a chain, *e.g.*:

```python
positions = espressomd.polymer.linear_polymer_positions(n_polymers=1,
                                                        beads_per_chain=10,
                                                        bond_length=1, seed=42,
                                                        min_distance=0.9)
```

which would create positions for a single polymer with 10 monomers. Please check the documentation for a more detailed description.

Furthermore we want to compute the diffusion constant of the polymer for different numbers of monomers.
For this purpose we can again use the multiple tau correlator. The following script computes the mean
squared displacement for the center of mass of the polymer as well as the average hydrodynamic radius
$R_h$, end-to-end distance $R_F$ and radius of gyration $R_g$.

How do $R_h$, $R_g$, $R_F$ and the diffusion coefficient $D$ depend on the number of monomers?
You can refer to the Flory theory of polymers, and assume you are simulating a real polymer in a
good solvent, with Flory exponent $\nu \approx 0.588$.

In [ ]:
import logging
import sys

import numpy as np

import espressomd
import espressomd.accumulators
import espressomd.observables
import espressomd.polymer

logging.basicConfig(level=logging.INFO, stream=sys.stdout)

espressomd.assert_features(['LENNARD_JONES'])

# Setup constants
TIME_STEP = 0.01
LOOPS = 4000
STEPS = 100

# System setup
system = espressomd.System(box_l=[12.0, 12.0, 12.0])
system.cell_system.skin = 0.4

# Lennard-Jones interaction
system.non_bonded_inter[0, 0].lennard_jones.set_params(
    epsilon=1.0, sigma=1.0, shift="auto", cutoff=2.0**(1.0 / 6.0))

# Fene interaction
fene = espressomd.interactions.FeneBond(k=7, r_0=1, d_r_max=2)
system.bonded_inter.add(fene)

N_MONOMERS = np.array([6, 8, 10])

tau_results = []
msd_results = []
rh_results = []
rf_results = []
rg_results = []
for index, N in enumerate(N_MONOMERS):
    logging.info("Polymer size: {}".format(N))
    # create a linear polymer with Fene bonds
    positions = espressomd.polymer.linear_polymer_positions(n_polymers=1,
                                                            beads_per_chain=N,
                                                            bond_length=1, seed=42,
                                                            min_distance=0.9)
    for i, pos in enumerate(positions[0]):
        pid = len(system.part)
        system.part.add(id=pid, pos=pos)
        if i > 0:
            system.part[pid].add_bond((fene, pid - 1))

    logging.info("Warming up the polymer chain.")
    system.time_step = 0.002
    system.integrator.set_steepest_descent(
        f_max=1.0,
        gamma=10,
        max_displacement=0.01)
    system.integrator.run(2000)
    system.integrator.set_vv()
    logging.info("Warmup finished.")

    logging.info("Equilibration.")
    system.time_step = TIME_STEP
    system.thermostat.set_langevin(kT=1.0, gamma=50, seed=42)
    system.integrator.run(2000)
    logging.info("Equilibration finished.")

    system.thermostat.turn_off()

    lbf = espressomd.lb.LBFluidGPU(
        kT=1,
        seed=42,
        agrid=1,
        dens=1,
        visc=5,
        tau=TIME_STEP)
    system.actors.add(lbf)
    system.thermostat.set_lb(LB_fluid=lbf, seed=42, gamma=5)

    logging.info("Warming up the system with LB fluid.")
    system.integrator.run(1000)
    logging.info("Warming up the system with LB fluid finished.")

    # configure correlator
    com_pos = espressomd.observables.ComPosition(ids=range(N))
    correlator = espressomd.accumulators.Correlator(
        obs1=com_pos, tau_lin=16, tau_max=LOOPS * STEPS, delta_N=1,
        corr_operation="square_distance_componentwise", compress1="discard1")
    system.auto_update_accumulators.add(correlator)

    logging.info("Sampling started.")
    rhs = np.zeros(LOOPS)
    rfs = np.zeros(LOOPS)
    rgs = np.zeros(LOOPS)
    for i in range(LOOPS):
        system.integrator.run(STEPS)
        rhs[i] = system.analysis.calc_rh(
            chain_start=0,
            number_of_chains=1,
            chain_length=N)[0]
        rfs[i] = system.analysis.calc_re(
            chain_start=0,
            number_of_chains=1,
            chain_length=N)[0]
        rgs[i] = system.analysis.calc_rg(
            chain_start=0,
            number_of_chains=1,
            chain_length=N)[0]

    logging.info("Sampling finished.")

    # store results
    correlator.finalize()
    corrdata = correlator.result()
    tau = correlator.lag_times()
    msd = np.sum(corrdata, axis=1)
    tau_results.append(tau)
    msd_results.append(msd)
    rh_results.append(rhs)
    rf_results.append(rfs)
    rg_results.append(rgs)

    # reset system
    system.part.clear()
    system.thermostat.turn_off()
    system.actors.clear()
    system.auto_update_accumulators.clear()

rh_results = np.array(rh_results)
rf_results = np.array(rf_results)
rg_results = np.array(rg_results)
tau_results = np.array(tau_results)
msd_results = np.reshape(msd_results, [len(N_MONOMERS),-1])

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
plt.rcParams.update({'font.size': 22})

In [ ]:
import scipy.signal

def autocorrelation(time_series):
    '''
    Normalized autocorrelation function with zero-padding. This is a
    scipy implementation of the algorithm described in de Buyl 2018,
    JOSS 3(28), pp. 877 (https://doi.org/10.21105/joss.00877).
    '''
    n = time_series.size
    n_with_padding = 2**int(np.ceil(np.log2(n)) + 1)
    signal = np.zeros(n_with_padding)
    signal[:n] = time_series
    acf = scipy.signal.correlate(signal, signal, mode='full', method='fft')[-n_with_padding:][:n]
    acf_normalized = acf / (n - np.arange(n))  # rescale by bin sizes
    return acf_normalized

def standard_error_mean_autocorrelation(time_series, variable_label):
    '''
    Calculate the mean and the correlation-corrected standard error
    of the mean of time series by integrating the autocorrelation
    function. Due to the short simulation length, it is not possible
    to fit an exponential to the long-time tail. Instead, return a
    percentile.
    '''
    summary = []
    fig = plt.figure(figsize=(10, 6))
    for signal, N in zip(time_series, N_MONOMERS):
        acf = autocorrelation(signal - np.mean(signal))
        # the acf cannot be integrated beyond tau=N/2
        integral = np.array([acf[0] + 2 * np.sum(acf[1:j]) for j in np.arange(1, len(acf)//2)])
        # remove the noisy part of the integral
        negative_number_list = np.nonzero(integral < 0)
        if negative_number_list[0].size:
            integral = integral[:int(0.95 * negative_number_list[0][0])]
        # compute the standard error of the mean
        std_err = np.sqrt(integral / acf.size)
        # due to the small sample size, the long-time tail is not
        # well resolved and cannot be fitted, so we use a percentile
        asymptote = np.percentile(std_err, 75)
        # plot the integral and asymptote
        p = plt.plot([0, len(std_err)], 2 * [asymptote], '--')
        plt.plot(np.arange(len(std_err)) + 1, std_err,
                 '-', color=p[0].get_color(),
                 label=f'$\\int {variable_label}$ for N={N}')
        summary.append((np.mean(signal), asymptote))
    plt.xlabel('Lag time $\\tau / \\Delta t$')
    plt.ylabel(f'$\\int_{{-\\tau}}^{{+\\tau}} {variable_label}$')
    plt.legend()
    plt.show()
    return np.array(summary)

Plot the end-to-end distance $R_F$ of the polymer as a function of the number of monomers. What relation do you observe?

The end-to-end distance follows the law $R_F = c_F N^\nu$ with $c_F$ a constant and $\nu$ the Flory exponent.

In [ ]:
rf_summary = standard_error_mean_autocorrelation(rf_results, '\\operatorname{acf}(R_F)')
rf_exponent, rf_prefactor = np.polyfit(np.log(N_MONOMERS), np.log(rf_summary[:,0]), 1)
rf_prefactor = np.exp(rf_prefactor)

fig = plt.figure(figsize=(10, 8))
x = np.linspace(min(N_MONOMERS) - 0.5, max(N_MONOMERS) + 0.5, 20)
plt.plot(x, rf_prefactor * x**rf_exponent, '-',
         label=f'$R_F^{{\\mathrm{{fit}}}} = {rf_prefactor:.2f} N^{{{rf_exponent:.2f}}}$')
plt.errorbar(N_MONOMERS, rf_summary[:,0],
             yerr=rf_summary[:,1],
             ls='', marker='o', capsize=5, capthick=1,
             label='$R_F^{\\mathrm{simulation}}$')
plt.xlabel('Number of monomers $N$')
plt.ylabel('End-to-end distance [$\sigma$]')
plt.legend()
plt.show()

Plot the radius of gyration $R_g$ of the polymer as a function of the number of monomers. What relation do you observe?

The radius of gyration follows the law $R_g = c_g N^\nu$ with $c_g$ a constant and $\nu$ the Flory exponent.

In [ ]:
rg_summary = standard_error_mean_autocorrelation(rg_results, '\\operatorname{acf}(R_g)')
rg_exponent, rg_prefactor = np.polyfit(np.log(N_MONOMERS), np.log(rg_summary[:,0]), 1)
rg_prefactor = np.exp(rg_prefactor)

fig = plt.figure(figsize=(10, 8))
x = np.linspace(min(N_MONOMERS) - 0.5, max(N_MONOMERS) + 0.5, 20)
plt.plot(x, rg_prefactor * x**rg_exponent, '-',
         label=f'$R_g^{{\\mathrm{{fit}}}} = {rg_prefactor:.2f} N^{{{rg_exponent:.2f}}}$')
plt.errorbar(N_MONOMERS, rg_summary[:,0],
             yerr=rg_summary[:,1],
             ls='', marker='o', capsize=5, capthick=1,
             label='$R_g^{\\mathrm{simulation}}$')
plt.xlabel('Number of monomers $N$')
plt.ylabel('Radius of gyration [$\sigma$]')
plt.legend()
plt.show()

Plot the hydrodynamic radius $R_h$ of the polymers as a function of the number of monomers. What relation do you observe?

The hydrodynamic radius can be calculated via the Stokes radius, i.e. the radius of a sphere that
diffuses at the same rate as the polymer. An approximative formula is $R_h \approx c_h N^{1/3}$
with $c_h$ a constant.

In [ ]:
rh_summary = standard_error_mean_autocorrelation(rh_results, '\\operatorname{acf}(R_h)')
rh_exponent, rh_prefactor = np.polyfit(np.log(N_MONOMERS), np.log(rh_summary[:,0]), 1)
rh_prefactor = np.exp(rh_prefactor)

fig = plt.figure(figsize=(10, 8))
x = np.linspace(min(N_MONOMERS) - 0.5, max(N_MONOMERS) + 0.5, 20)
plt.plot(x, rh_prefactor * x**rh_exponent, '-',
         label=f'$R_h^{{\\mathrm{{fit}}}} = {rh_prefactor:.2f} N^{{{rh_exponent:.2f}}}$')
plt.errorbar(N_MONOMERS, rh_summary[:,0],
             yerr=rh_summary[:,1],
             ls='', marker='o', capsize=5, capthick=1,
             label='$R_h^{\\mathrm{simulation}}$')
plt.xlabel('Number of monomers $N$')
plt.ylabel('Hydrodynamic radius [$\sigma$]')
plt.legend()
plt.show()

Calculate the diffusion coefficient of the polymers.

In [ ]:
# cutoff for the diffusive regime (approximative)
tau_f_index = 40
# cutoff for the data series (larger lag times have larger variance due to undersampling)
tau_max_index = 81

plt.figure(figsize=(10, 10))
plt.xlabel(r'$\tau$ [$\Delta t$]')
plt.ylabel(r'MSD [$\sigma^2$]')
for index, (tau, msd) in enumerate(zip(tau_results, msd_results)):
    plt.loglog(tau[1:120], msd[1:120], label=r"$N=${}".format(N_MONOMERS[index]))
plt.loglog(2 * [tau[tau_f_index]], [0, np.max(msd_results)], '-', color='black')
plt.text(tau[tau_f_index], np.max(msd_results), r'$\tau_{f}$')
plt.loglog(2 * [tau[tau_max_index]], [0, np.max(msd_results)], '-', color='black')
plt.text(tau[tau_max_index], np.max(msd_results), r'$\tau_{max}$')
plt.legend()
plt.show()

In [ ]:
diffusion_results = np.zeros(len(N_MONOMERS))
plt.figure(figsize=(10, 8))
plt.xlabel(r'$\tau$ [$\Delta t$]')
plt.ylabel(r'MSD [$\sigma^2$]')
for index, (tau, msd) in enumerate(zip(tau_results, msd_results)):
    a, b = np.polyfit(tau[tau_f_index:tau_max_index], msd[tau_f_index:tau_max_index], 1)
    x = np.array([tau[1], tau[tau_max_index - 1]])
    p = plt.plot(x, a * x + b, '-')
    plt.plot(tau[1:tau_max_index], msd[1:tau_max_index], 'o', color=p[0].get_color(),
             label=r'$N=${}'.format(N_MONOMERS[index]))
    diffusion_results[index] = a / 6
plt.legend()
plt.show()

Recalling the formula for the diffusion coefficient of a short polymer in the Kirkwood–Zimm model:

$$D = \frac{D_0}{N} + \frac{k_B T}{6 \pi \eta} \left\langle \frac{1}{R_h} \right\rangle$$

where $D_0$ is the monomer diffusion coefficient and $\eta$ is the viscosity of the fluid.

In [ ]:
import scipy.optimize

def kirkwood_zimm(x, a, b, exponent):
    return a / x + b / x**exponent

(a, b), _ = scipy.optimize.curve_fit(
    lambda x, a, b: kirkwood_zimm(x, a, b, rh_exponent),
    N_MONOMERS, diffusion_results)

label = f'''\
$D^{{\\mathrm{{fit}}}} = \
    \\frac{{{a:.2f}}}{{N}} + \
    \\frac{{{b * 6 * np.pi:.2f} }}{{6\\pi}} \\cdot \
    \\frac{{{1}}}{{N^{{{rh_exponent:.2f}}}}}$ \
'''

fig = plt.figure(figsize=(10, 8))
x = np.linspace(min(N_MONOMERS) - 0.5, max(N_MONOMERS) + 0.5, 20)
plt.plot(x, kirkwood_zimm(x, a, b, rh_exponent), '-', label=label)
plt.plot(N_MONOMERS, diffusion_results, 'o', label='$D^{\\mathrm{simulation}}$')
plt.xlabel('Number of monomers $N$')
plt.ylabel('Diffusion coefficient')
plt.legend()
plt.show()

## References

[1] S. Succi. *The lattice Boltzmann equation for fluid dynamics and beyond.* Clarendon Press, Oxford, 2001.  
[2] B. Dünweg and A. J. C. Ladd. *Advanced Computer Simulation Approaches for Soft Matter Sciences III*, chapter II, pages 89–166. Springer, 2009.  
[3] B. Dünweg, U. Schiller, and A.J.C. Ladd. Statistical mechanics of the fluctuating lattice-Boltzmann equation. *Phys. Rev. E*, 76:36704, 2007.  
[4] P. G. de Gennes. *Scaling Concepts in Polymer Physics*. Cornell University Press, Ithaca, NY, 1979.  
[5] M. Doi. *Introduction to Polymer Physics.* Clarendon Press, Oxford, 1996.  
[6] Michael Rubinstein and Ralph H. Colby. *Polymer Physics.* Oxford University Press, Oxford, UK, 2003.  
[7] Daan Frenkel and Berend Smit. *Understanding Molecular Simulation.* Academic Press, San Diego, second edition, 2002.